# Rozpoznawanie jezyka programowania na podstawie fragmentu kodu

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np

file_path = 'data.csv'
df = pd.read_csv(file_path,encoding='utf-8',engine='python')

In [3]:
df.head()

,language,proj_id,file_id,file_body
0,JavaScript,10001,100001,// Functions as values of a variable\nvar cube...
1,JavaScript,10001,100002,// Functions as values of a variable\nvar cube...
2,JavaScript,10002,100003,function median(ary) {\n if (ary.length == ...
3,JavaScript,10002,100004,"[\n null,\n 4,\n 3.5,\n 2.1\n]\n"
4,JavaScript,10002,100005,(() => {\n 'use strict';\n\n // median :...


# Kodujemy jezyki za pomoca liczb

In [4]:
language_set=set(df['language'])
keys=dict()
i=0
for lang in language_set:
    keys.update({lang:i})
    i+=1

In [5]:
keys

{'Mathematica': 0,
 'Julia': 1,
 'PHP': 2,
 'R': 3,
 'Fortran': 4,
 'Ruby': 5,
 'MATLAB': 6,
 'Perl': 7,
 'Scala': 8,
 'Java': 9,
 'Python': 10,
 'C': 11,
 'Haskell': 12,
 'JavaScript': 13,
 'Swift': 14,
 'Rust': 15,
 'Kotlin': 16,
 'C++': 17,
 'Go': 18}

In [6]:
languages = df['language']
snipet = df['file_body']

label = []
text=[]
for lang,s in zip(languages,snipet):
        label.append(keys[lang])
        text.append(s)

label = np.array(label)
text = np.array(text)

In [7]:
label.shape

(16272,)

In [8]:
text.shape

(16272,)

# Podzial na dane testowe i treningowe

In [9]:
from sklearn.model_selection import train_test_split

x_train = []
x_test = []
y_train = []
y_test = []

x_train, x_test, y_train, y_test = train_test_split(text,label,train_size=0.9,shuffle=True,random_state=42)

x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

# Tokenizacja danych

In [10]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

count_vect = CountVectorizer(decode_error='ignore')
x_train_count = count_vect.fit_transform(x_train)
tfidf_trans = TfidfTransformer()
x_train_tfidf = tfidf_trans.fit_transform(x_train_count)

x_test_count = count_vect.transform(x_test)
x_test_tfidf = tfidf_trans.transform(x_test_count)

# Trenowanie modelu

In [11]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

clf = XGBClassifier(n_estimators=400)
clf.fit(x_train_tfidf,y_train)
y_pred = clf.predict(x_test_tfidf)
print(accuracy_score(y_test,y_pred))

0.8642506142506142


/home/filip/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


## Model trenuje sie okolo 10 minut. Daje zgodnosc w 86% przypadkow. Calkiem dobrze jak na proof of concept.

### Dalsze kroki:
1. Przetestowac inne algorytmy
2. Zrobic dokladniejszy tunning wybranego modelu
3. Stworzyc dodatkowe sposoby rozpoznawania jezyka przy uzyciu wyrazen regularnych i skladni charakterystycznych dla danego jezyka
4. Sprawdzic, czy da sie poprawic wynik wykorzystujac informacje o projekcie w ramach ktorego powstal kod